# Khipus.ai Hackathon 2025 - Universidad NUR
## Practico Final : ChatGPT - ITIL
### Team Itil           : Andres Moron - Javier Castellon Galvis - Huascar Rivero Lara

<span>© Copyright Notice 2025, Khipus.ai - All Rights Reserved.</span>

### Paso 1. Instalar dependencias necesarias

    ChromaDB                Es una base de datos vectorial open-source  
    sentence-transformers   Proporciona modelos preentrenados para generar embeddings de texto  
    PyPDF                   Es una biblioteca para manipular archivos PDF en Python     

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# # Este script inicializa un cliente ChromaDB, recupera una colección e imprime todos los documentos y sus metadatos, junto con el recuento de documentos en la colección.

from chromadb.utils import embedding_functions      # Para usar funciones de incrustación
from chromadb.config import Settings                # Para configurar el cliente ChromaDB
import chromadb                                     # Importar el cliente ChromaDB

# Inicializar el cliente ChromaDB con un directorio persistente
chroma_client = chromadb.Client(Settings(persist_directory="./chroma_db"))  # Configurar el directorio persistente
collection = chroma_client.get_or_create_collection("itil_docs")            # Recuperar o crear la colección llamada "itil_docs"

# Para ver todos los documentos y metadatos en la colección
all_data = collection.get()                         # Recuperar todos los documentos y metadatos de la colección
print(all_data)                                     # Imprimir todos los documentos y metadatos

#  Para contar el número de documentos en la colección
count = collection.count()                          # Contar el número de documentos en la colección    
print("\n[INFO]: El número de documentos en la colección 'itil_docs' es:", count)  # Imprimir el número de documentos en la colección   


{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': []}

[INFO]: El número de documentos en la colección 'itil_docs' es: 0


### Paso 2 : Chunking - Embeddings - Carga Vector Database (consistent)

    Chunking - Carga y divide documentos 
    Crea embeddings con Ollama 
    Carga en ChromaDB

In [ ]:
import os                                                                       # Importar las bibliotecas necesarias de LangChain y ChromaDB
from langchain_community.document_loaders import PyPDFLoader                    # Para cargar documentos PDF
from langchain_text_splitters import RecursiveCharacterTextSplitter             # Para dividir documentos en fragmentos
from langchain_community.embeddings import OllamaEmbeddings                     # Para crear embeddings con Ollama
from langchain_community.llms import Ollama                                     # Para usar el modelo LLM de Ollama    
from langchain_community.vectorstores import Chroma                             # Para usar ChromaDB como base de datos vectorial    
from langchain.chains import RetrievalQA                                        # Para crear una cadena de preguntas y respuestas (RAG)    

# Chunking - Carga y divide documentos ---
# La ruta del PDF se mantiene igual
pdf_path = "./docs/ITIL4Intro.pdf"
loader = PyPDFLoader(pdf_path)          # Carga el PDF usando PyPDFLoader
documents = loader.load()               # Carga los documentos del PDF

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)   # Configura el divisor de texto para dividir los documentos en fragmentos
docs = text_splitter.split_documents(documents)                                     # Divide los documentos en fragmentos
print(f"Documentos cargados: {len(documents)}. Fragmentos creados: {len(docs)}.")

# Crea embeddings con Ollama ---
# Usa el modelo de embeddings de Ollama, que por defecto usará 'llama2'
print("\n[INFO]: Creando embeddings con Ollama. Esto puede tardar mucho en CPU...")
embeddings = OllamaEmbeddings(model="llama2")                                       # Configura el modelo de embeddings de Ollama

# Carga en ChromaDB ---
# Crea o se conecta a la base de datos vectorial persistente
vectordb = Chroma.from_documents(                                                   # Crea la base de datos vectorial a partir de los documentos
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_db",
    collection_name="itil_docs"
)
print("[ÉXITO]: ChromaDB ha sido poblada con los embeddings.")

# Duracion 35 minutos

Documentos cargados: 30. Fragmentos creados: 178.

[INFO]: Creando embeddings con Ollama. Esto puede tardar mucho en CPU...


C:\Users\Yayo Rivero\AppData\Local\Temp\ipykernel_7284\117736389.py:22: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama2")


[ÉXITO]: ChromaDB ha sido poblada con los embeddings.


### Paso 3 : Configuración - Retriever - RAG

    Configura el LLM local con Ollama 
    Crea un retriever para buscar en la base de datos 
    Cadena de preguntas y respuestas
    Invoca la cadena RAG con la consulta

In [ ]:
# Configura el LLM local con Ollama ---
# Usa el modelo de Ollama 'llama2' para la generación de texto
print("[INFO]: Conectando con el LLM de Ollama...")
llm = Ollama(model="llama2")                # Configura el modelo LLM de Ollama
print("[ÉXITO]: LLM de Ollama listo.")

# Crea el pipeline RAG y haz una pregunta ---
# Crea un retriever para buscar en la base de datos
retriever = vectordb.as_retriever()         # Configura el retriever para buscar en la base de datos vectorial

# Crea la cadena de preguntas y respuestas (RAG) ---
# Crea la cadena RAG con el LLM y el retriever
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)    # Configura la cadena de preguntas y respuestas

# Haz una consulta ---  
print("\n[INFO]: Preparando para hacer una consulta a la IA...")
# Haz una consulta
query = "¿Qué es ITIL?"
print(f"\n[INFO]: Haciendo la consulta: '{query}'")
response = qa_chain.invoke({"query": query})        # Invoca la cadena RAG con la consulta
print("\nRespuesta de la IA:")
print(response['result'])

[INFO]: Conectando con el LLM de Ollama...
[ÉXITO]: LLM de Ollama listo.

[INFO]: Preparando para hacer una consulta a la IA...

[INFO]: Haciendo la consulta: '¿Qué es ITIL?'

Respuesta de la IA:
ITIL stands for Information Technology Infrastructure Library. It is a framework of guidelines and best practices for IT service management, which helps organizations to manage their IT infrastructure and deliver high-quality services to their customers. The ITIL framework covers four dimensions of service management - Service Strategy, Service Design, Service Transition, and Service Operation - and provides a structure for organizing and managing IT services in a way that meets the needs of the organization and its customers.


### Paso Extra : Consulta a la DB Vectorial desde 0

    Configura el modelo de embeddings de Ollama
    Carga la base de datos vectorial desde el directorio persistente
    Inicializar el modelo LLM de Ollama
    Crear el retriever y la cadena QA
    Realizar una pregunta

In [ ]:
# --- IMPORTS NECESARIOS ---
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA

# --- CONFIGURACIÓN ---
CHROMA_PATH = "./chroma_db"
COLLECTION_NAME = "itil_docs"
OLLAMA_MODEL = "llama2"  # Puedes usar otro modelo si tienes: "mistral", "phi", etc.

# --- PASO 1: Crear los embeddings ---
print("[INFO]: Inicializando embeddings de Ollama...")
embeddings = OllamaEmbeddings(model=OLLAMA_MODEL)       # Configura el modelo de embeddings de Ollama

# --- PASO 2: Cargar la base de datos vectorial existente ---
print("[INFO]: Cargando base de datos Chroma desde disco...")
vectordb = Chroma(                                      # Carga la base de datos vectorial desde el directorio persistente
    persist_directory=CHROMA_PATH,
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME
)
print("[ÉXITO]: Base de datos Chroma cargada.")

# --- PASO 3: Inicializar el modelo LLM de Ollama ---
print("[INFO]: Conectando con el LLM de Ollama...")
llm = Ollama(model=OLLAMA_MODEL)                        # Configura el modelo LLM de Ollama
print("[ÉXITO]: LLM de Ollama listo.")

# --- PASO 4: Crear el retriever y la cadena QA ---
retriever = vectordb.as_retriever()                     # Configura el retriever para buscar en la base de datos vectorial
qa_chain = RetrievalQA.from_chain_type(                 # Configura la cadena de preguntas y respuestas
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# --- PASO 5: Realizar una pregunta ---
query = "¿Qué sabes del motor de un auto Corolla Cross? Respóndeme en español."
print(f"\n[Consulta]: {query}")
respuesta = qa_chain.run(query)
print(f"\n[Respuesta]:\n{respuesta}")



[INFO]: Inicializando embeddings de Ollama...
[INFO]: Cargando base de datos Chroma desde disco...
[ÉXITO]: Base de datos Chroma cargada.
[INFO]: Conectando con el LLM de Ollama...
[ÉXITO]: LLM de Ollama listo.

[Consulta]: ¿Qué sabes del motor de un auto Corolla Cross? Respóndeme en español.


C:\Users\Yayo Rivero\AppData\Local\Temp\ipykernel_3280\332879851.py:16: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=OLLAMA_MODEL)
C:\Users\Yayo Rivero\AppData\Local\Temp\ipykernel_3280\332879851.py:20: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(
C:\Users\Yayo Rivero\AppData\Local\Temp\ipykernel_3280\332879851.py:29: LangChainDepr


[Respuesta]:
I don't know anything about the Toyota Corolla Cross engine as it is not related to the context provided in the passage. The passage discusses ITIL, a framework for continuous service improvement, and its components such as Plan-Do-Check-Act and the importance of creating value for customers. It does not mention anything about cars or their engines.
